<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S75_drone_decision_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U gtbook

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import plotly.express as px
import gtsam 

from gtbook.discrete import Variables
from gtbook.display import show, pretty

# Trajectory Optimization

> We can use factor graphs to optimize over future trajectories, as well.

<img src="Figures7/S75-Autonomous_camera_drone-05.jpg"  alt="Splash image with steampunk drone in aggressive maneuver" width="40%" align=center style="vertical-align:middle;margin:10px 0px">


In the previous section we saw how do use factor graphs for visual SLAM and structure from motion. These perception algorithms are typically run after we have gathered some visual information, and inform us about what happened in the past. But how can we plan for the future? We already saw that RRTs are useful paradigm to plan in a continuous, potentially high dimensional state space. 

In this section we will discuss a different method, which is optimal control. In particular, we will use **trajectory optimization**, to minimize the cost associated with the trajectory that we would like to execute in the future. These costs can be associated with staying away from obstacles, or other desirable properties like minimizing power consumption to preserve battery life. Finally, RRTs and trajectory optimization can be combined, where RRT finds a good "broad strokes" trajectory, and trajectory optimization smooths out and fine-tunes the final trajectory.

**Jan '23: This section is still under construction, please check back later.**